In [1]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib
%pip install accelerate

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-zdh3w9nn
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-zdh3w9nn
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit ecf72c716f6378a9bb2191ed884b5ce9428dcc81
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path 
import gc    
import torch
import os
import torch
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%cd /root/mesh-transformer

/root/mesh-transformer


In [4]:
from accelerate import notebook_launcher
def training_function(num_processes=2):
    autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,   
        codebook_size = 2048, 
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8, 
        attn_decoder_depth  = 4,
        attn_encoder_depth = 2) 
    
    pkg = torch.load("./checkpoints/mesh-autoencoder.ckpt.epoch_0_avg_loss_-0.06490_recon_0.3518_commit_-0.8334.pt")  
    autoencoder.load_state_dict(pkg['model'], strict = False) 
    
    dataset = MeshDataset.load("./objverse_250f_490.7M_all_17561_labels_568425_5_min_x5_aug.npz")  
    dataset2 = MeshDataset.load("./objverse_250f_98.1M_all_17561_labels_113685_5_min_x1_aug.npz")
    dataset.data.extend(dataset2.data)  
    dataset2 = MeshDataset.load("./shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
    dataset.data.extend(dataset2.data)  
    dataset2 = MeshDataset.load("./shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
    dataset.data.extend(dataset2.data) 
    dataset.sort_dataset_keys() 
         
    autoencoder.commit_loss_weight = 0.5
    autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                                 batch_size=32,
                                                 grad_accum_every =2,
                                                 learning_rate = 1e-3,
                                                 checkpoint_every_epoch=1)  
    _loss1 = autoencoder_trainer.train(14445,  diplay_graph= False)        
 


In [5]:
args = ()
notebook_launcher(training_function, args, num_processes=4)

Launching training on 4 GPUs.
[MeshDataset] Loaded 568425 entrys
[MeshDataset] Created from 568425 entrys
[MeshDataset] Loaded 568425 entrys
[MeshDataset] Created from 568425 entrys
[MeshDataset] Loaded 568425 entrys
[MeshDataset] Created from 568425 entrys
[MeshDataset] Loaded 568425 entrys
[MeshDataset] Created from 568425 entrys
[MeshDataset] Loaded 113685 entrys
[MeshDataset] Created from 113685 entrys
[MeshDataset] Loaded 113685 entrys
[MeshDataset] Created from 113685 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 113685 entrys
[MeshDataset] Created from 113685 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 113685 entrys
[MeshDataset] Created from 113685 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 21560 entrys
[MeshDataset] Created from 21560 entrys


Epoch 1/14445:   0%|          | 1/5514 [00:01<2:30:11,  1.63s/it, commit_loss=-0.804, loss=-0.0283, recon_loss=0.374][rank0]:[W reducer.cpp:1389] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
[rank2]:[W reducer.cpp:1389] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forwa

ChildFailedError: 
============================================================
training_function FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2024-06-12_21:00:02
  host      : 5f2a0b06308c
  rank      : 2 (local_rank: 2)
  exitcode  : 1 (pid: 943)
  error_file: /tmp/torchelastic_iocikef3/none_ie5fenih/attempt_0/2/error.json
  traceback : Traceback (most recent call last):
    File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 347, in wrapper
      return f(*args, **kwargs)
    File "/tmp/ipykernel_504/2028672340.py", line 32, in training_function
      _loss1 = autoencoder_trainer.train(14445,  diplay_graph= False)
    File "/usr/local/lib/python3.10/dist-packages/meshgpt_pytorch/trainer.py", line 344, in train
      for batch_idx, batch in progress_bar: